In [2]:
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import plotly.graph_objects as go

In [4]:
# Load the CSV file
data_file = r'C:\Users\User\Desktop\UCLA\fall quarter\COMPTNG 16B\target_and_wholefoods.csv'
df = pd.read_csv(data_file)

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Grocery Price Comparison"

# Initialize a dictionary to store accumulated prices
accumulated_prices = {"Target": 0, "Wholefoods": 0}

# Define the layout
app.layout = html.Div([
    html.H1("Grocery Price Comparison", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Choose a Category:"),
        dcc.Dropdown(
            id="category-dropdown",
            options=[{"label": cat, "value": cat} for cat in df["Category"].unique()],
            placeholder="Select a category"
        ),
    ], style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Label("Search for a Product:"),
        dcc.Input(id="product-search", type="text", placeholder="Enter product name..."),
        html.Button("Search", id="search-button", n_clicks=0),
    ], style={'margin-bottom': '20px'}),
    
    html.Div(id="product-results", style={'margin-bottom': '20px'}),
    
    html.Div(id="price-comparison", style={'margin-bottom': '20px'}),
    
    html.Div([
        dcc.Graph(id="price-comparison-chart")
    ])
])

In [5]:
# Define callback to update product results
@app.callback(
    Output("product-results", "children"),
    Input("search-button", "n_clicks"),
    State("category-dropdown", "value"),
    State("product-search", "value"),
    prevent_initial_call=True
)
def search_products(n_clicks, category, product_name):
    if not category or not product_name:
        return "Please select a category and enter a product name."
    
    # Filter data
    filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False))]
    
    if filtered_df.empty:
        return "No products found."
    
    # Sort by price-to-quantity ratio
    sorted_target = filtered_df[filtered_df["Store"] == "Target"].sort_values("Price/Quantity")
    sorted_wholefoods = filtered_df[filtered_df["Store"] == "Wholefoods"].sort_values("Price/Quantity")
    
    # Generate results
    target_list = html.Ul([html.Li(f"{row['Name']} - ${row['Price/Quantity']} per unit") for _, row in sorted_target.iterrows()])
    wholefoods_list = html.Ul([html.Li(f"{row['Name']} - ${row['Price/Quantity']} per unit") for _, row in sorted_wholefoods.iterrows()])
    
    return html.Div([
        html.H3("Target:"),
        target_list,
        html.H3("Wholefoods:"),
        wholefoods_list
    ])


In [6]:
# Define callback to update total prices and comparison
@app.callback(
    [Output("price-comparison", "children"),
     Output("price-comparison-chart", "figure")],
    Input("search-button", "n_clicks"),
    State("category-dropdown", "value"),
    State("product-search", "value"),
    prevent_initial_call=True
)
def update_total_prices(n_clicks, category, product_name):
    if not category or not product_name:
        return "", go.Figure()
    
    # Filter data
    filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False))]
    
    if filtered_df.empty:
        return "No products found.", go.Figure()
    
    # Accumulate prices for the first item in each store (assuming sorted by lowest price)
    target_price = filtered_df[filtered_df["Store"] == "Target"]["Price/Quantity"].min()
    wholefoods_price = filtered_df[filtered_df["Store"] == "Wholefoods"]["Price/Quantity"].min()
    
    if pd.notna(target_price):
        accumulated_prices["Target"] += target_price
    if pd.notna(wholefoods_price):
        accumulated_prices["Wholefoods"] += wholefoods_price
    
    # Create comparison chart
    fig = go.Figure(data=[
        go.Bar(name="Target", x=["Total Cost"], y=[accumulated_prices["Target"]]),
        go.Bar(name="Wholefoods", x=["Total Cost"], y=[accumulated_prices["Wholefoods"]])
    ])
    fig.update_layout(title="Total Price Comparison", barmode="group")
    
    # Display total prices
    comparison_text = f"Total Price: Target - ${accumulated_prices['Target']:.2f}, Wholefoods - ${accumulated_prices['Wholefoods']:.2f}"
    return comparison_text, fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)